# Classification task - Room occupancy 1

The goal of this task is to predict a room occupancy based on Temperature, Humidity, Light and CO2 measurements. Ground-truth occupancy was obtained from time stamped pictures that were taken every minute.

## Data source
[http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+](http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+)

## Feature description
* **Date** - time stamp in the following format: year-month-day hour:minute:second 
* **Temperature** - temperature in degrees of Celsius 
* **Relative Humidity** - Relative humidity in % 
* **Light** - light intensity in Lux 
* **CO2** - amount of CO2 in the air, measured in ppm 
* **Humidity Ratio** - Humidity ratio derived from temperature and relative humidity, in kgwater-vapor/kg-air 
* **Occupancy** - a target binary value, 0 for not occupied, 1 for occupied status

In [0]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/mlcollege/introduction-to-ml/master/data/occupancy.csv', sep=',')
data.head()

## Simple classifier
Implement a simple classifier based on all numerical features.

### Data preparation

In [0]:
from sklearn.model_selection import train_test_split

# Prepare the data by separating features and target variable
X_all = data[['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']]
y_all = data['Occupancy']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y_all,
    random_state=1,
    test_size=0.1)

print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

### Training a classifier

Train a classifier using the following models:
* [Gaussian Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
* [Logistic regression](http://scikit-learn.org/0.15/modules/generated/sklearn.linear_model.LogisticRegression.html)
* [Support Vector Machines](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) (experiment with different kernels)
* [Gradient Boosted Trees](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (Experiment with different depths and number of trees)

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Train a classifier - uncomment the model you want to use
# Try different models and hyperparameters
clf = GaussianNB()
#clf = LogisticRegression()
#clf = SVC(kernel='linear')
#clf = GradientBoostingClassifier(n_estimators=50)
clf.fit(X_train, y_train)

### Evaluate the models

Implement all evaluation methods you have learned in the Scikit-learn tutorial. Decide which model performs best on the given problem.

In [0]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

# Evaluate the model on the test set
y_pred = clf.predict(X_test)

print ("Test accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))
print ()
print(metrics.classification_report(y_test, y_pred))

In [0]:
# Evaluate the model on the training set
y_pred = clf.predict(X_train)

print ("Train accuracy: {:.2f}".format(accuracy_score(y_train, y_pred)))
print ()
print(metrics.classification_report(y_train, y_pred))